In [83]:
import matplotlib.pyplot as plt
import numpy as np
from moviepy.editor import *
from PIL import Image


# SETTINGS
dimensionality = 10
size = 8


clip = VideoFileClip("videos/0HH3N4OZUXUB.mp4")
#test.ipython_display()

hashes = np.array([])
for t in np.linspace(0, clip.duration, dimensionality):
    im = Image.fromarray(clip.get_frame(t), "RGB")
    im = im.resize((size, size), Image.ANTIALIAS)
    red, green, blue = im.split()

    pixels_r = list(red.getdata())
    pixels_g = list(green.getdata())
    pixels_b = list(blue.getdata())
    
    avg_r = sum(pixels_r)/len(pixels_r)
    avg_g = sum(pixels_g)/len(pixels_g)
    avg_b = sum(pixels_b)/len(pixels_b)
    
    bits_r = "".join(map(lambda pixel_r: '1' if pixel_r < avg_r else '0', pixels_r))
    bits_g = "".join(map(lambda pixel_g: '1' if pixel_g < avg_g else '0', pixels_g))
    bits_b = "".join(map(lambda pixel_b: '1' if pixel_b < avg_b else '0', pixels_b))
    
    hashes = np.append(hashes, (int(bits_r, 2).__format__('016x').upper()))
    hashes = np.append(hashes, (int(bits_g, 2).__format__('016x').upper()))
    hashes = np.append(hashes, (int(bits_b, 2).__format__('016x').upper()))


print(hashes)



['FFBF80E0F701009F' 'FF0080C0FFC381FF' '000080C0FFE7C1FF'
 'FFBF80FCF5810081' 'FF0080FCFF8701FF' '000080FDFFEF81FF'
 'FF00FFC1C3830001' '0000FFC1C387008F' '0000FFC1E787008F'
 'BF00808000FFFFFF' '0000808103FFFFFF' '000080C1E7FFFFFF'
 'FF81808183000FFF' '018080C1C7009FFF' '000080C3EFC19FFF'
 'FF8D8081C30000FF' '858080C1E7819FFF' '008080C1FFC19FFF'
 'FF8D80C0C30001FF' 'BF8080C0E7818FFF' '000080C0FFC18FFF'
 'FFBF8080C70180BF' 'FF0080C0FFC181FF' '000080C0FFE3C1FF'
 'FFFF00809F0100BF' 'FF0080C0FFF3B3FF' '000080C0FFF7F3FF'
 'FF9F8080070038FF' 'BF008080BFB1FBFF' '000080C0FFF3FBFF']
